In [1]:
import pandas as pd
import math
import os
import tqdm
import numpy as np
#import bamboolib as bam
import sklearn
from matplotlib import pyplot 
import seaborn as sns
from sklearn import metrics
from matplotlib import pyplot as plt
    
import re
import scipy

# Définition de l'espace de travail

In [2]:
aymeric = "/home/aymeric/python-scripts/anses_medialab/datas" #aymeric
jp = 'd:/Projects/Medialab/Anses/' # Jean Philippe
path= aymeric
tfidf='tfidf'
#domain='media'
#domain='twitter'
#domain='facebook'
path_file= f'{aymeric}/fb_tw_med_data.csv.gz'

## Chargement des fichiers

In [3]:
## Chargement des trois corpus concaténés tels que sauvegarder dans le fichier fb_tw_med_data.csv.gz
## au besoin le fichier est ici : 
# https://drive.google.com/file/d/1xAjeMsgJudbX2l7GOM7Fo9ECj93lx-aA/view?usp=share_link

df = pd.read_csv(path_file, dtype ={"id":str}).drop_duplicates() #Vérifier si besoin de définir les types

df["new_id"] = df.index #On créer un nouvel id à partir de l'index dans le dataframe
df = df.drop(columns=["Unnamed: 0"]) #on supprime la colonne inutile "Unnamed: 0"
df.columns #le nom des colonne


Index(['id', 'date', 'text', 'source', 'origin', 'new_id'], dtype='object')

In [4]:
# On charge la liste des termes avec leurs clusters d'appartenance. La colonne utilisée est celle qui s'appelle nom_cluster.3
clst = pd.read_csv('/home/aymeric/python-scripts/anses_medialab/analyse_mixte/node_list_for_gephi.csv', sep=",").drop(columns=["Unnamed: 0", "Id"])
clst

,Label,name_cluster,nom_cluster,cluster.y,nom_cluster.1,nom_cluster.2,nom_cluster.3
0,agriculture biologique,Luttes contre les nuisibles,Viticulture et pesticides,15,Viticulture et pesticides,Viticulture et pesticides,Le vin
1,bordelais,Luttes contre les nuisibles,Viticulture et pesticides,15,Viticulture et pesticides,Viticulture et pesticides,Le vin
2,sulfites,Luttes contre les nuisibles,Viticulture et pesticides,15,Viticulture et pesticides,Viticulture et pesticides,Le vin
3,traitements phytosanitaires,Luttes contre les nuisibles,Viticulture et pesticides,15,Viticulture et pesticides,Viticulture et pesticides,Le vin
4,vignerons,Luttes contre les nuisibles,Viticulture et pesticides,15,Viticulture et pesticides,Viticulture et pesticides,Le vin
...,...,...,...,...,...,...,...
448,espace,Expositions des riverains,Zéro phyto,10,Plan écophyto,Régulation des pesticides,Plan écophyto
449,espaces verts,Expositions des riverains,Zéro phyto,10,Plan écophyto,Régulation des pesticides,Plan écophyto
450,expériences,Développement durable,Zéro phyto,10,Plan écophyto,Régulation des pesticides,Plan écophyto
451,expérimentation,Développement durable,Zéro phyto,10,Plan écophyto,Régulation des pesticides,Plan écophyto


In [11]:
# On charge la liste des requêtes "représentatives" des clusters
# Dans ce fichier on a aussi une colonne qui précise le type de configuration temporelle (episodic or chronic)
# et la dimension (environment or health)
queries = pd.read_csv(f"{aymeric}/search_queries/list_of_queries.csv", sep =",")

# Recherche des motifs

In [13]:
#On parcours la liste des formes à rechercher et on la met dans un dictionnaire avec comme key le nom du cluster 
#et comme value la requête
dict_queries ={}
for n, x in enumerate(queries["nom_cluster.3"]):
    liste_regle = []
    sq1= queries["query_1"].iloc[n]
    liste_regle.append(sq1)
    sq2= queries["query_2"].iloc[n]
    if isinstance(sq2, str):
        liste_regle.append(sq2)
    else:
        pass
    dict_queries[x] = liste_regle
dict_queries

{'Le vin': ['vigne|viticult'],
 'Plan écophyto': ['écophyto|zéro phyto(s*)'],
 'Transition écologique et débats publics': ['débat(s*) public(s*)|convention(s*) citoyenne(s*)|transition.*écologique(s*)'],
 'Épandages et riverains': ['épandage|pulvéris(ations*)', 'riverain|distance'],
 'Le rapport du CIRC': ['\\bcirc\\b|(centre|centre international) .*recherche.*cancer|\\biarc\\b'],
 'SDHI': ['sdhi'],
 'Affaire Triskalia': ['triskalia|nutréa|nutrea|(nutréa||nutrea)-triskalia'],
 'Les espèces nuisibles': ['insecte|parasite|ravageur', 'traitement(s*)'],
 'Le virus Zika': ['zika'],
 'Chlordécone': ['chlordécone'],
 'La mortalité des abeilles': ['abeille|apicult'],
 'Néonicotinoïdes': ['(néo|neo)-?nic(otino(ï|i)de*)|(néo|neo)-?nic(s*)'],
 'Procès de Monsanto': ['monsanto'],
 'La science en question': ['science|scientifique'],
 'Politiques agricoles françaises et sortie du glyphosate': ['Travert|sortie du glypho'],
 'La crise de la biodiversité': ['biodiversité'],
 'Perturbateurs endocriniens

In [14]:
# Pour éviter de travailler avec un dataframe, 
#je mets les texts dans un dictionnaire avec comme key le "new_id" et la value le "text"

list_id = []
list_text = []
dict_text= {}
for n, x in enumerate(df['new_id']):
    list_id.append(x)
    list_text.append(df["text"].iloc[n])
    dict_text[x]= df["text"].iloc[n]

In [15]:
#On utilisera ce nouveau dataframe pour le fusionner avec le résultat de la recherche
data = {"new_id": list_id, "text": list_text}
df0 = pd.DataFrame(data)
df0

,new_id,text
0,0,"@EmmanuelMacron Transition agroécologique, ou ..."
1,1,#Béthune : un fumigène insecticide alerte les ...
2,2,Alors je récapitule :\nLa lavande c'est danger...
3,3,#RappelProduit\nPREPAUSE 60 comprimés - LABORA...
4,4,#ESG Les politiques d’exclusions appliquées co...
...,...,...
1816026,1816026,La boutique de l'Arbalou - Soins Arbalou\nsite...
1816027,1816027,Offre de stage rénumérée : Réalisation d'un fi...
1816028,1816028,Good Smile Company News Roundup!\nFriday!! ...
1816029,1816029,Bioconsomacteurs.org - actus\n\nbioconsomacteu...


In [16]:
# La boucle pour recherche les expressions dans les textes
dict_compteur = {}
for x in dict_queries:
    dict_test_queries = {}
    query = dict_queries[x][0]
    p = re.compile(query, re.IGNORECASE)
    if len(dict_queries[x])==2: # Si le nombre de groupe d'expression à recherche est supérieur à 2 on créer un second compileur
        query2 = dict_queries[x][1]
        p2 = re.compile(query2, re.IGNORECASE)
        
    else:
        pass
    compteur = 0 # utiliser pour compter le nombre de requêtes retournée "vraies". 
    for y in dict_text:
        #Les résultats sont placés dans un dictionnaire 
        #avec comme key le "new_id" et la value le motif trouvé (si vrai) ou np.nan (si faux)
        text = dict_text[y]
        sq = p.search(text)
        if sq is not None:
            if len(dict_queries[x]) == 2 :
                sq2 = p2.search(text)
                if sq2 is not None: #si la première condition et vraie et qu'il y a deux groupes d'expressions
                    dict_test_queries[y] = f"{sq.group()} {sq2.group()}"
                    compteur +=1
                else:
                    dict_test_queries[y] = np.nan
            else:
                dict_test_queries[y] = sq.group()
                compteur +=1
        else:
            dict_test_queries[y] = np.nan
    dict_compteur[x]= compteur
    df0[x] = df0["new_id"].map(dict_test_queries) # on fusionne le dictionnaire contenant le résultat avec le dataframe défini dans la cellue précédente
    print(x, compteur)
    

Le vin 30804
Plan écophyto 8392
Transition écologique et débats publics 14069
Épandages et riverains 9333
Le rapport du CIRC 7663
SDHI 7712
Affaire Triskalia 1988
Les espèces nuisibles 4569
Le virus Zika 3202
Chlordécone 39927
La mortalité des abeilles 156146
Néonicotinoïdes 122201
Procès de Monsanto 116760
La science en question 107699
Politiques agricoles françaises et sortie du glyphosate 14800
La crise de la biodiversité 46906
Perturbateurs endocriniens 20379
Pollution de l'eau 7831
Sécurité des aliments 9551
Réchauffement climatique 22612
Régulation européenne 17707
Troubles de santé 78110


In [17]:
# On fait une nouvelle fusion de df0 avec df (dataframe de départ) 
# pour récupérer quelques méta-données (date, origin, source)

df1 = df0.merge(df[['id', 'date', 'source', 'origin', 'new_id']], on = ["new_id"], how ="left")
df1['date'] = pd.to_datetime(df1['date'], infer_datetime_format=True)
df1['yearmonth']=(df1['date'].dt.strftime('%Y-%m'))
df1['year']=(df1['date'].dt.strftime('%Y'))

# Calcule des fréquences

In [18]:
#Pour sauvegarder le df1
df1.to_csv(f"{aymeric}/search_queries/corpus_after_search_queries.csv", sep =",")

In [ ]:
#Pour lire le df1
df1 = pd.read_csv(f"{aymeric}/search_queries/corpus_after_search_queries.csv", sep =",")
df1 = df1.drop(columns=["Unnamed: 0"])
df1.columns

In [20]:
origin = False
echelle = "yearmonth" #Or "year"
if origin == True :
    if echelle == "yearmonth":
        df_freq = df1.groupby(['origin', 'year','yearmonth']).agg(nb_txt = ('new_id', 'size')).reset_index()
        for x in dict_queries:
            print(x)
            d = df1.loc[~df1[x].isnull()].groupby(['origin','year',"yearmonth"]).agg(nb_doc = (x, "size")).reset_index()
            d = d.rename(columns = {"nb_doc":x})
            df_freq = df_freq.merge(d, on = ['origin','year',"yearmonth"], how = "left")
        df_freq1 = df_freq.copy()#[['yearmonth', 'origin', "nb_txt", "Le vin", "Plan écophyto"]]
        df_freq1 = df_freq1.melt(id_vars=['year', 'yearmonth',"nb_txt"])
        df_freq1 = df_freq1.rename(columns={"variable":"theme"})
        df_freq1["freq"] = df_freq1["value"]/df_freq1["nb_txt"]
        df_freq1["year2"] = pd.to_datetime(df_freq1.year, format='%Y')
        df_freq1["yearmonth2"] = pd.to_datetime(df_freq1.yearmonth, format='%Y-%m')
    elif echelle == "year":
        df_freq = df1.groupby(['origin', 'year']).agg(nb_txt = ('new_id', 'size')).reset_index()
        for x in dict_queries:
            print(x)
            d = df1.loc[~df1[x].isnull()].groupby(['origin','year']).agg(nb_doc = (x, "size")).reset_index()
            d = d.rename(columns = {"nb_doc":x})
            df_freq = df_freq.merge(d, on = ['origin','year'], how = "left")
        df_freq1 = df_freq.copy()
        df_freq1 = df_freq1.melt(id_vars=['year', "nb_txt"])
        df_freq1 = df_freq1.rename(columns={"variable":"theme"})
        df_freq1["freq"] = df_freq1["value"]/df_freq1["nb_txt"]
        df_freq1["year2"] = pd.to_datetime(df_freq1.year, format='%Y')
else:
    if echelle == "yearmonth":
        df_freq = df1.groupby(['year','yearmonth']).agg(nb_txt = ('new_id', 'size')).reset_index()
        for x in dict_queries:
            print(x)
            d = df1.loc[~df1[x].isnull()].groupby(['year',"yearmonth"]).agg(nb_doc = (x, "size")).reset_index()
            d = d.rename(columns = {"nb_doc":x})
            df_freq = df_freq.merge(d, on = ['year',"yearmonth"], how = "left")
        df_freq1 = df_freq.copy()#[['yearmonth', 'origin', "nb_txt", "Le vin", "Plan écophyto"]]
        df_freq1 = df_freq1.melt(id_vars=['year', 'yearmonth',"nb_txt"])
        df_freq1 = df_freq1.rename(columns={"variable":"theme"})
        df_freq1["freq"] = df_freq1["value"]/df_freq1["nb_txt"]
        df_freq1["year2"] = pd.to_datetime(df_freq1.year, format='%Y')
        df_freq1["yearmonth2"] = pd.to_datetime(df_freq1.yearmonth, format='%Y-%m')
    elif echelle == "year":
        df_freq = df1.groupby(['year']).agg(nb_txt = ('new_id', 'size')).reset_index()
        for x in dict_queries:
            print(x)
            d = df1.loc[~df1[x].isnull()].groupby(['year']).agg(nb_doc = (x, "size")).reset_index()
            d = d.rename(columns = {"nb_doc":x})
            df_freq = df_freq.merge(d, on = ['year'], how = "left")
        df_freq1 = df_freq.copy()
        df_freq1 = df_freq1.melt(id_vars=['year', "nb_txt"])
        df_freq1 = df_freq1.rename(columns={"variable":"theme"})
        df_freq1["freq"] = df_freq1["value"]/df_freq1["nb_txt"]
        df_freq1["year2"] = pd.to_datetime(df_freq1.year, format='%Y')


Le vin
Plan écophyto
Transition écologique et débats publics
Épandages et riverains
Le rapport du CIRC
SDHI
Affaire Triskalia
Les espèces nuisibles
Le virus Zika
Chlordécone
La mortalité des abeilles
Néonicotinoïdes
Procès de Monsanto
La science en question
Politiques agricoles françaises et sortie du glyphosate
La crise de la biodiversité
Perturbateurs endocriniens
Pollution de l'eau
Sécurité des aliments
Réchauffement climatique
Régulation européenne
Troubles de santé


In [ ]:
#df_freq = pd.DataFrame(data = {"origin":["facebook", "media", "twitter"]}).reset_index()




In [ ]:
df_freq1 = df_freq.copy()#[['yearmonth', 'origin', "nb_txt", "Le vin", "Plan écophyto"]]
df_freq1 = df_freq1.melt(id_vars=['year', 'yearmonth',"nb_txt"])
df_freq1 = df_freq1.rename(columns={"variable":"theme"})
df_freq1["freq"] = df_freq1["value"]/df_freq1["nb_txt"]
df_freq1["year2"] = pd.to_datetime(df_freq1.year, format='%Y')

In [ ]:
df_freq1["yearmonth2"] = pd.to_datetime(df_freq1.yearmonth, format='%Y-%m')

In [21]:
df_freq1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3234 entries, 0 to 3233
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year        3234 non-null   object        
 1   yearmonth   3234 non-null   object        
 2   nb_txt      3234 non-null   int64         
 3   theme       3234 non-null   object        
 4   value       2677 non-null   float64       
 5   freq        2677 non-null   float64       
 6   year2       3234 non-null   datetime64[ns]
 7   yearmonth2  3234 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(3)
memory usage: 202.2+ KB


In [22]:
df_freq1.to_csv(f"{aymeric}/search_queries/freq_of_queries.csv", sep =",")
#df = pd.read_csv(f"{aymeric}/search_queries/corpus_after_search_queries.csv", sep =",")

In [23]:
df_freq1["year"] = df_freq1["year"].astype('int64')
df_freq1 = df_freq1.loc[(df_freq1["year"]>=2010)]

# Création d'un tableau avec des sparklines

In [24]:
import base64
import requests
from time import sleep
from itertools import chain
from io import BytesIO
from datetime import timedelta, date
from IPython.display import display, HTML

In [25]:
# Turn off the max column width so the HTML 
# image tags don't get truncated 
pd.set_option('display.max_colwidth', None)

# Turning off the max column will display all the data in
# our arrays so limit the number of element to display
pd.set_option('display.max_seq_items', 2)

In [26]:
def sparkline(data, figsize=(4, 0.25), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    data = list(data)
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])    

    plt.plot(len(data) - 1, data[len(data) - 1], 'r.')

    ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)
    
    img = BytesIO()
    plt.savefig(img)
    img.seek(0)
    plt.close()
    return '<img src="data:image/png;base64,{}"/>'.format(base64.b64encode(img.read()).decode())

In [27]:
if "origin" in df_freq1.columns:
    for n, y in enumerate(df_freq1['origin'].unique()):
        print(n)
        data = []
        df_temp = df_freq1.loc[df_freq1["origin"]==y]

        for m, x in enumerate(df_temp["theme"].unique()):
            df_temp2 = df_temp.loc[df_temp["theme"]==x].sort_values("yearmonth2")
            list_freq = [x for x in df_temp2["freq"]]

            data.append((x,  list_freq))
        spark0 = pd.DataFrame(data, columns=['theme', 'data'])
        spark0[f'sparklines_{y}'] = spark0.data.map(sparkline)
        if n == 0:
            print(n)
            spark = spark0.copy()
        else:
            spark = spark.merge(spark0, on = ["theme"], how = "left")
    queries1 = queries.rename(columns={"nom_cluster.3":"theme"})
    queries2 = queries1.merge(spark, on = ["theme"], how= "left")
    queries2
    spark = queries2[["theme", "Theme", "Configuration", "sparklines_facebook", "sparklines_twitter", "sparklines_media"]]
    # _repr_html_ escapes HTML so manually handle the rendering

else:
    data = []
    df_temp = df_freq1.copy()
    for m, x in enumerate(df_temp["theme"].unique()):
        df_temp2 = df_temp.loc[df_temp["theme"]==x].sort_values("yearmonth2")
        list_freq = [x for x in df_temp2["freq"]]

        data.append((x,  list_freq))
    spark = pd.DataFrame(data, columns=['theme', 'data'])
    spark['sparklines'] = spark.data.map(sparkline)
    queries1 = queries.rename(columns={"nom_cluster.3":"theme"})
    queries2 = queries1.merge(spark, on = ["theme"], how= "left")
    queries2
    spark = queries2[["theme", "Theme", "Configuration", "sparklines"]].sort_values("Configuration")
    # _repr_html_ escapes HTML so manually handle the rendering

    


#years = [x for x in df_freq1["year2"].unique()]
data

[('Le vin',
  [nan,
   nan,
   0.002044989775051125,
   0.011267605633802818,
   0.008403361344537815,
   0.001584786053882726,
   0.0035971223021582736,
   nan,
   0.011799410029498525,
   nan,
   0.0027548209366391185,
   nan,
   0.07454289732770746,
   nan,
   0.006020066889632107,
   0.0036330608537693005,
   0.013422818791946308,
   0.011583011583011582,
   0.003575685339690107,
   0.005235602094240838,
   0.034324942791762014,
   0.024738344433872503,
   0.0051440329218107,
   0.013774104683195593,
   0.008110300081103,
   0.006419753086419753,
   0.0022185246810870773,
   0.010675129832660127,
   0.014380918975797966,
   0.019711902956785442,
   0.006961506961506962,
   0.0025329280648429585,
   0.00140597539543058,
   0.013175584834632966,
   0.009763617677286743,
   0.0028169014084507044,
   0.0022411474675033617,
   0.06541389153187441,
   0.0044797849703214244,
   0.008047373215912541,
   0.008119079837618403,
   0.007262164124909223,
   0.004549283909014322,
   0.0042063937

In [28]:
spark1 = spark[["Theme", "Configuration", "sparklines"]].sort_values("Configuration")
HTML(spark1.to_html(escape=False))


,Theme,Configuration,sparklines
10,Bees’ Decline,Chronic,
2,Ecological transition and public debates,Chronic,
7,Pests,Chronic,
21,Health troubles,Chronic,
20,European regulation of pesticides,Chronic,
11,Neonicotinoide,Chronic,
4,IARC’s report,Chronic,
13,Science in debate,Chronic,
15,Biodiversity Crisis,Chronic,
17,Water pollution,Chronic,


In [29]:
list_of_lists = {}
for n, x in enumerate(clst["nom_cluster.3"].unique()):
    df_temp = clst.loc[clst["nom_cluster.3"]==x]
    list_in_str = " ; ".join(y for y in df_temp["Label"].unique()[0:5])
    list_of_lists[x] = list_in_str


In [30]:
spark["terms"] = spark["theme"].map(list_of_lists)
spark2 = spark[["Theme", "terms", "Configuration", "sparklines"]].sort_values("Configuration")
HTML(spark2.to_html(escape=False))


,Theme,terms,Configuration,sparklines
10,Bees’ Decline,abeilles ; apiculteur ; apiculture ; colonies d'abeilles ; essaims,Chronic,
2,Ecological transition and public debates,assemblée ; assemblée nationale ; campagne ; citoyens ; climat,Chronic,
7,Pests,alternative écologique ; animal ; animal domestique ; bicarbonate de soude ; colmar,Chronic,
21,Health troubles,agents pathogènes ; algue ; alzheimer ; cnrs ; docteur,Chronic,
20,European regulation of pesticides,autorisation ; bruxelles ; commission ; commission environnement ; commission européenne,Chronic,
11,Neonicotinoide,betterave ; betteraves sucrières ; confédération paysanne ; dérogation ; filière,Chronic,
4,IARC’s report,agence ; autorité européenne ; avis ; cancérogène ; cancérogène probable,Chronic,
13,Science in debate,arguments ; article ; biotechnology ; chercheuse ; conclusion,Chronic,
15,Biodiversity Crisis,abeilles sauvages ; chauves-souris ; disparition ; écosystèmes ; effondrement,Chronic,
17,Water pollution,cours d'eau ; cultures de maïs ; eau ; eau potable ; maïs,Chronic,
